In [1]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import requests
import json
from selenium.webdriver import Chrome
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import time
from datetime import datetime

In [2]:
driver = Chrome(executable_path='/Users/chrisjohnson/Downloads/chromedriver')
url = "https://sheraton.marriott.com/destinations/"
driver.get(url)
time.sleep(30)
viewhotels_button = driver.find_elements_by_xpath('/html/body/div[1]/div[1]/div[4]/div[4]/a')[0]
viewhotels_button.click()
# northamhotels_button = driver.find_elements_by_xpath('/html/body/section/pacsys-animation[3]/pacsys-directory/div[2]/pacsys-directory-listings/div/div[5]/a')[0]
# northamhotels_button.click()

In [3]:
page_source = driver.page_source
soup = BeautifulSoup(page_source, 'lxml')

In [4]:
stuff = soup.find_all('a', {'class', 'property'})
# stuff
hotel_list = []
for x in stuff:
    hotel = {}
    hotel['name'] = x.text.strip()
    hotel['url'] = x.attrs['href']
    hotel_list.append(hotel)
hotelsurl_df = pd.DataFrame(hotel_list)
hotelsurl_df

,name,url
0,Sheraton Club des Pins Resort,https://www.marriott.com/ALGSI
1,Sheraton Annaba Hotel,https://www.marriott.com/AAESI
2,Sheraton Oran Hotel,https://www.marriott.com/ORNSI
3,Sheraton Djibouti,https://www.marriott.com/JIBSI
4,Sheraton Montazah Hotel,https://www.marriott.com/ALYSI
...,...,...
458,Sheraton Quito Hotel,https://www.marriott.com/UIOSI
459,Sheraton Asuncion Hotel,https://www.marriott.com/ASUSI
460,Sheraton Lima Hotel & Convention Center,https://www.marriott.com/LIMSI
461,Sheraton Colonia Golf & Spa Resort,https://www.marriott.com/CYRSI


In [5]:
%%time

hotel_list = []
counter = 0
for index, row in hotelsurl_df.iterrows():
    name = row['name']
    url = row['url']
    try:
        res = requests.get(url)
        soup = BeautifulSoup(res.content, 'lxml')
        stuff = soup.find('div', {'class' : 'l-container l-header-container'})
#             hotel_list = []
        hotel = {}
        try:
            hotel['brand'] = 'Sheraton'
        except:
            hotel['brand'] = 'Sheraton'
        try:
            hotel['name'] = name
        except:
            hotel['name'] = name
        try:
            hotel['url'] = url
        except:
            hotel['url'] = url
        try:
            hotel['street'] = stuff.find_all('span')[2].text
        except:
            hotel['street'] = 'N/A'
        try:
            hotel['locality'] = stuff.find_all('span')[4].text
        except:
            hotel['locality'] = 'N/A'        
        try:
            hotel['state'] = stuff.find_all('span')[6].text
        except:
            hotel['state'] = 'N/A'            
        try:
            hotel['postalcode'] = stuff.find_all('span')[7].text
        except:
            hotel['postalcode'] = 'N/A'            
        try:
            hotel['coordinate1'] = stuff.find_all('span')[25].text
        except:
            hotel['coordinate1'] = 'N/A'            
        try:
            hotel['coordinate2'] = stuff.find_all('span')[26].text
        except:
            hotel['coordinate2'] = 'N/A'
        try:
            hotel['coordinate3'] = stuff.find_all('span')[27].text
        except:
            hotel['coordinate3'] = 'N/A'            
        try:
            hotel['coordinate4'] = stuff.find_all('span')[28].text
        except:
            hotel['coordinate4'] = 'N/A'            
        
        desc = soup.find_all('p', {'class' : 'l-margin-top-none l-overflow-hidden t-collapsed-height'})
        try:
            hotel['description'] = str(desc).replace('<p class="l-margin-top-none l-overflow-hidden t-collapsed-height">','').replace('</p>', '')
        except:
            hotel['description'] = 'N/A'
        
        overallratings = stuff = soup.find('div', {'class' : 'l-overall-rating-container'})
        try:
            hotel['overall_rating'] = ratings.find_all('div')[0].text.strip()[:4]
        except:
            hotel['overall_rating'] = 'N/A'
        
        specratings = stuff = soup.find('div', {'class' : 'l-secondary-ratings-container l-clear l-margin-bottom-quarter l-m-margin-bottom-none'})
        try:
            hotel['cleanliness_rating'] = specratings.find_all('div')[2].text.strip()[:4]
        except:
            hotel['cleanliness_rating'] = 'N/A'
        try:
            hotel['service_rating'] = specratings.find_all('div')[7].text.strip()[:4]
        except:
            hotel['service_rating'] = 'N/A'
            
        hotel_list.append(hotel)
        print(hotel)
        print()
        print(datetime.now())
        print('='*80)
        print('='*80)
        print()
#         driver.close()
#         driver.switch_to.window(original_window)
#         time.sleep(1)

        counter += 1
        print(f'Collected {counter} hotels.')
    except:
        hotel = {}
        hotel['name'] = name
        hotel['url'] = url
        hotel_list.append(hotel)
        print(f"Number {x} didn't work at {datetime.now()}")
        continue
hotel_df = pd.DataFrame(hotel_list)
hotel_df     

{'brand': 'Sheraton', 'name': 'Sheraton Club des Pins Resort', 'url': 'https://www.marriott.com/ALGSI', 'street': 'Boite Postal 62, Club Des Pins, Staoueli', 'locality': 'Algiers', 'state': '', 'postalcode': '16101', 'coordinate1': 'Reviews', 'coordinate2': 'https://www.marriott.com/hotels/maps/travel/algsi-sheraton-club-des-pins-resort/?directPageRequest=true', 'coordinate3': '36.763332', 'coordinate4': '2.873333', 'description': "[ Escape to the Mediterranean Sea at Sheraton Club des Pins Resort, a 5-star Algeria beachfront hotel with indulgent amenities in a prime Algiers setting. Unwind in our spacious rooms and suites, each with thoughtful touches and views of the sea. Or lounge in our waterfront gardens before strolling over to our private beach for a day in the beautiful sunshine of Algeria. Relax alongside the sea, or enjoy activities at Club Nautique. Go for a swim in our pools, play tennis and volleyball, take a fitness class or be pampered at our incredible Algiers hotel spa

{'brand': 'Sheraton', 'name': 'Sheraton Cairo Hotel & Casino', 'url': 'https://www.marriott.com/CAISI', 'street': 'Galae Square, P. O. Box 11', 'locality': 'Cairo', 'state': '', 'postalcode': '', 'coordinate1': 'Reviews', 'coordinate2': ' View Rates ', 'coordinate3': '', 'coordinate4': ' View Rates ', 'description': '[ Located on the banks of the River Nile, Sheraton Cairo Hotel &amp; Casino offers effortless service and modern accommodations in the heart of Cairo, Egypt. Select from a variety of dining options, including 24-hour room service and our popular Italian restaurant. If you are looking to host a business meeting or special event, our hotel has an array of inspiring and flexible spaces, complete with the latest technology and bespoke planning services. Take a dive into our outdoor pool or get active in our 24-hour fitness center. Rejuvenate in our spa with its wide variety of treatments. All of our rooms and suites offers stunning views of Cairo, some featuring private balcon

{'brand': 'Sheraton', 'name': 'Sheraton Casablanca Hotel & Towers', 'url': 'https://www.marriott.com/CMNSI', 'street': '100, Avenue des F.A.R.', 'locality': 'Casablanca', 'state': '', 'postalcode': '20000', 'coordinate1': 'https://www.marriott.com/hotels/maps/travel/cmnsi-sheraton-casablanca-hotel-and-towers/?directPageRequest=true', 'coordinate2': '33.596319', 'coordinate3': '-7.612109', 'coordinate4': 'N/A', 'description': '[ Come together with everything you need to escape and explore the White City, Casablanca. Ideally located, in the heart of the City, at only 35 minutes away from the Mohamed V International airport (CMN) and walking distance to the Old Medina, Central Market, Art Deco buildings and all facilities from train stations, restaurants to mall center, making Sheraton Casablanca Hotel &amp; Towers as the perfect jumping-off point for joining your business or exploring the City. All the essentials for an effortless stay, for every occasion, from business to leisure and fa

{'brand': 'Sheraton', 'name': 'Sheraton Kampala Hotel', 'url': 'https://www.marriott.com/EBBSI', 'street': 'Ternan Avenue, P. O. Box 7041', 'locality': 'Kampala', 'state': '', 'postalcode': '', 'coordinate1': 'Reviews', 'coordinate2': 'https://www.marriott.com/hotels/maps/travel/ebbsi-sheraton-kampala-hotel/?directPageRequest=true', 'coordinate3': '0.3167', 'coordinate4': '32.5833', 'description': "[ The Sheraton Kampala Hotel is ideally located in the heart of Kampala city and welcomes travellers with a relaxed ambience and a dazzling array of upscale amenities. Just 45 minutes' drive from Entebbe International Airport, we are within reach of the Central Business District. Take in a magnificent view of the city from our spacious and comfortable guest rooms and suites, homely decorated with comfort in mind. Take in sweeping views of the Kampala skyline from the comfort of our vibrant, high-style hotel surrounded by 9 acres of well-manicured gardens. Stay fit at our state-of-the-art Fit

{'brand': 'Sheraton', 'name': 'Sheraton Grand Beijing Dongcheng Hotel', 'url': 'https://www.marriott.com/BJSDO', 'street': '36 North Third Ring Road East, Dongcheng District', 'locality': 'Beijing', 'state': '', 'postalcode': '100013', 'coordinate1': 'Reviews', 'coordinate2': 'https://www.marriott.com/hotels/maps/travel/bjsdo-sheraton-grand-beijing-dongcheng-hotel/?directPageRequest=true', 'coordinate3': '39.9671', 'coordinate4': '116.40832', 'description': '[ Conveniently located on Beijing’s North Third Ring Road with close proximity to tourist and business districts, the Sheraton Grand Beijing Dongcheng Hotel offers 439 spacious guest rooms, including 70 suites. Just steps from Global Trade Center and a convenient trip from locales such as Tiananmen Square and the Forbidden City, our hotel is the ideal choice for business and leisure travelers. Our contemporary rooms feature the brand’s Sheraton Signature Sleep Experience Bed, a flat-screen TV and more. In your downtime, experience 

...

{'brand': 'Sheraton', 'name': 'Sheraton Madison Hotel', 'url': 'https://www.marriott.com/MSNSI', 'street': '706 John Nolen Drive', 'locality': 'Madison', 'state': 'Wisconsin', 'postalcode': '53713', 'coordinate1': 'https://www.marriott.com/hotels/maps/travel/msnsi-sheraton-madison-hotel/?directPageRequest=true', 'coordinate2': '43.047238', 'coordinate3': '-89.374548', 'coordinate4': 'N/A', 'description': '[ Explore Wisconsin’s capital city. A few steps from our doors, enjoy a walk or bike ride on the Capital City Bike Trail. A scenic, short trip to downtown along Lake Monona, takes you to State Street. With a collection of local shops, State Street offers visitors a wide array of restaurants and unique shopping. Find a quiet recharge in our modern guest rooms and suites. Showcasing thoughtful touches like our Signature Sheraton Sleep Experience Bed, high-speed Wi-Fi, complimentary parking and plenty of power connections to keep your portable devices charged, you will feel connected whi

{'brand': 'Sheraton', 'name': 'Sheraton Salta Hotel', 'url': 'https://www.marriott.com/SLASS', 'street': 'Coronel Francisco de Uriondo 330', 'locality': 'Salta', 'state': '', 'postalcode': 'A4406BKQ', 'coordinate1': '-65.398383', 'coordinate2': 'N/A', 'coordinate3': 'N/A', 'coordinate4': 'N/A', 'description': "[ Elevate your stay to new heights at Sheraton Salta Hotel. Our convenient location on the east end of the city affords us unrivaled access to a wealth of renowned attractions, such as Plaza 9 de Julio, the 20 de Febrero Monument and the Museum of High-Altitude Archaeology. After your day in the city, retire to the modern comfort of our spacious rooms and suites, boasting amazing views, marble bathrooms and flat-panel TVs. Savor a mouthwatering dinner at Restaurant Terracota for a gourmet selection of international cuisine and fine wine. For lighter fare, Jardin del Sol, our snack bar, features a menu of small plates, as well as craft cocktails. Alleviate any stress from a long d

{'brand': 'Sheraton', 'name': 'Sheraton Sao Paulo WTC Hotel', 'url': 'https://www.marriott.com/SAOSI', 'street': 'Nacoes Unidas Ave 12559, Brooklin Novo', 'locality': 'Sao Paulo', 'state': '', 'postalcode': '04578-903', 'coordinate1': 'Reviews', 'coordinate2': 'https://www.marriott.com/hotels/maps/travel/saosi-sheraton-sao-paulo-wtc-hotel/?directPageRequest=true', 'coordinate3': '-23.608014', 'coordinate4': '-46.697545', 'description': "[ Sheraton São Paulo WTC Hotel offers accommodations in Sao Paulo. The hotel has an outdoor pool and fitness center, and guests can enjoy a drink at the bar. Private parking is available on site.All rooms include a flat-screen TV. Each room is fitted with a private bathroom. For your comfort, you will find free toiletries and a hairdryer. There is a 24-hour front desk, hairdresser's, and gift shop at the building. Free use of bicycles is available at this hotel and the area is popular for biking.  The hotel also provides car rental. Faria Lima Avenue x 

{'brand': 'Sheraton', 'name': 'Sheraton Asuncion Hotel', 'url': 'https://www.marriott.com/ASUSI', 'street': 'Aviadores Del Chaco 2066', 'locality': 'Asuncion', 'state': '', 'postalcode': '', 'coordinate1': 'https://www.marriott.com/hotels/maps/travel/asusi-sheraton-asuncion-hotel/?directPageRequest=true', 'coordinate2': '-25.283948', 'coordinate3': '-57.564703', 'coordinate4': 'N/A', 'description': "[ Perfect for rest and relaxation, business endeavors or both, Sheraton Asuncion Hotel is located in the vivacious Financial District. The area is rich in parks and entertainment, placing you near other popular attractions. You can enjoy convenient access to Centro de Artes Visuales Museo Del Barro, Paseo La Galería and Silvio Pettirossi International Airport. When it's time to cater to your desire for R &amp; R, be still and schedule a massage at Neptune Spa. Following a pampering session, stop at our rooftop bar, Terraza, for tapas, cocktails and views of Paraguay. For a complete dining e

,brand,name,url,street,locality,state,postalcode,coordinate1,coordinate2,coordinate3,coordinate4,description,overall_rating,cleanliness_rating,service_rating
0,Sheraton,Sheraton Club des Pins Resort,https://www.marriott.com/ALGSI,"Boite Postal 62, Club Des Pins, Staoueli",Algiers,,16101,Reviews,https://www.marriott.com/hotels/maps/travel/al...,36.763332,2.873333,[ Escape to the Mediterranean Sea at Sheraton ...,N/A,3.8,3.4
1,Sheraton,Sheraton Annaba Hotel,https://www.marriott.com/AAESI,Boulevard Victor Hugo,Annaba,,23000,https://www.marriott.com/hotels/maps/travel/aa...,36.902294,7.762364,N/A,[ A transformative fixture within the city lan...,N/A,4.3,4.0
2,Sheraton,Sheraton Oran Hotel,https://www.marriott.com/ORNSI,"Route des Falaises Avenue Canastel, Seddikia",Oran,,,Reviews,https://www.marriott.com/hotels/maps/travel/or...,35.715464,-0.60913,[ Experience exceptional services and amenitie...,N/A,3.8,3.7
3,Sheraton,Sheraton Djibouti,https://www.marriott.com/JIBSI,Plateau de serpent,Plateau de Serpent,,1924,https://www.marriott.com/hotels/maps/travel/ji...,11.602457,43.157591,N/A,[ The Sheraton Djibouti is located on Plateau ...,N/A,3.3,3.3
4,Sheraton,Sheraton Montazah Hotel,https://www.marriott.com/ALYSI,Corniche Road,Alexandria,,21923,Reviews,View Rates,,View Rates,[ Sheraton Montazah Hotel welcomes you to Alex...,N/A,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
458,Sheraton,Sheraton Quito Hotel,https://www.marriott.com/UIOSI,Avenida Naciones Unidas y Avenida Republica de...,Quito,,,https://www.marriott.com/hotels/maps/travel/ui...,-0.177626,-78.478678,N/A,"[ Located on the 11th floor, Sheraton Fitness ...",N/A,4.8,4.7
459,Sheraton,Sheraton Asuncion Hotel,https://www.marriott.com/ASUSI,Aviadores Del Chaco 2066,Asuncion,,,https://www.marriott.com/hotels/maps/travel/as...,-25.283948,-57.564703,N/A,"[ Perfect for rest and relaxation, business en...",N/A,4.5,4.5
460,Sheraton,Sheraton Lima Hotel & Convention Center,https://www.marriott.com/LIMSI,Paseo de la Republica 170,Lima,,1,-12.057284,-77.036905,N/A,N/A,[ Business and leisure travelers will enjoy a ...,N/A,4.6,4.6
461,Sheraton,Sheraton Colonia Golf & Spa Resort,https://www.marriott.com/CYRSI,Continuación de la Rambla de Las Américas S/N,Colonia Del Sacramento,,70000,-34.435797,-57.872543,N/A,N/A,[ Elevate your travel experience with world-cl...,N/A,4.3,4.4


In [6]:
hotel_df = hotel_df.drop_duplicates(subset='name')

In [7]:
hotel_df

,brand,name,url,street,locality,state,postalcode,coordinate1,coordinate2,coordinate3,coordinate4,description,overall_rating,cleanliness_rating,service_rating
0,Sheraton,Sheraton Club des Pins Resort,https://www.marriott.com/ALGSI,"Boite Postal 62, Club Des Pins, Staoueli",Algiers,,16101,Reviews,https://www.marriott.com/hotels/maps/travel/al...,36.763332,2.873333,[ Escape to the Mediterranean Sea at Sheraton ...,N/A,3.8,3.4
1,Sheraton,Sheraton Annaba Hotel,https://www.marriott.com/AAESI,Boulevard Victor Hugo,Annaba,,23000,https://www.marriott.com/hotels/maps/travel/aa...,36.902294,7.762364,N/A,[ A transformative fixture within the city lan...,N/A,4.3,4.0
2,Sheraton,Sheraton Oran Hotel,https://www.marriott.com/ORNSI,"Route des Falaises Avenue Canastel, Seddikia",Oran,,,Reviews,https://www.marriott.com/hotels/maps/travel/or...,35.715464,-0.60913,[ Experience exceptional services and amenitie...,N/A,3.8,3.7
3,Sheraton,Sheraton Djibouti,https://www.marriott.com/JIBSI,Plateau de serpent,Plateau de Serpent,,1924,https://www.marriott.com/hotels/maps/travel/ji...,11.602457,43.157591,N/A,[ The Sheraton Djibouti is located on Plateau ...,N/A,3.3,3.3
4,Sheraton,Sheraton Montazah Hotel,https://www.marriott.com/ALYSI,Corniche Road,Alexandria,,21923,Reviews,View Rates,,View Rates,[ Sheraton Montazah Hotel welcomes you to Alex...,N/A,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
458,Sheraton,Sheraton Quito Hotel,https://www.marriott.com/UIOSI,Avenida Naciones Unidas y Avenida Republica de...,Quito,,,https://www.marriott.com/hotels/maps/travel/ui...,-0.177626,-78.478678,N/A,"[ Located on the 11th floor, Sheraton Fitness ...",N/A,4.8,4.7
459,Sheraton,Sheraton Asuncion Hotel,https://www.marriott.com/ASUSI,Aviadores Del Chaco 2066,Asuncion,,,https://www.marriott.com/hotels/maps/travel/as...,-25.283948,-57.564703,N/A,"[ Perfect for rest and relaxation, business en...",N/A,4.5,4.5
460,Sheraton,Sheraton Lima Hotel & Convention Center,https://www.marriott.com/LIMSI,Paseo de la Republica 170,Lima,,1,-12.057284,-77.036905,N/A,N/A,[ Business and leisure travelers will enjoy a ...,N/A,4.6,4.6
461,Sheraton,Sheraton Colonia Golf & Spa Resort,https://www.marriott.com/CYRSI,Continuación de la Rambla de Las Américas S/N,Colonia Del Sacramento,,70000,-34.435797,-57.872543,N/A,N/A,[ Elevate your travel experience with world-cl...,N/A,4.3,4.4


In [8]:
hotel_df.to_csv('sheraton_description.csv')